# Code for generating MIMIC postgres database and extracting pressor data
- Ruoyi Jiang
- July 14, 2019
- Build PGsql database

In [40]:
import os
import pandas as pd
import glob
import subprocess
import shlex

from sqlalchemy.sql import text
from sqlalchemy import create_engine, inspect
from sqlalchemy_utils import database_exists, create_database

# download psycopg2, sqlalchemy, sqlalchemy-utils
# python3 -m pip install sqlalchemy-utils --user

In [37]:
def createCommandSQL(sql_command_file):

    # Open the .sql file
    sql_file = open(sql_command_file,'r')

    # Create an empty command string
    sql_command = ''

    # Iterate over all lines in the sql file
    for line in sql_file:
        # Ignore commented lines
        if not line.startswith('--') and line.strip('\n'):
            # Append line to the command string
            sql_command += line.strip('\n').strip('\t')
    
    if sql_command.endswith(';'):
        return sql_command
    else:
        return None

### Build database (schema -> tables -> indexes)

In [31]:
# specify a base_dir 
base_dir = os.path.join(os.getenv("HOME"), "Projects/Haimovich_Pressors/collection/mimic")

In [32]:
os.makedirs(base_dir, exist_ok=True)

In [22]:
save_dir = os.path.join(os.getenv("HOME"), "Projects/Haimovich_Pressors/collection/out")

In [29]:
os.makedirs(save_dir, exist_ok=True)

wget the csv files

In [7]:
os.chdir(base_dir)

In [9]:
%%bash

# wget from MIMIC into base_dir (run from terminal, needs password input)
wget --user ruoyi.jiang@yale.edu --ask-password -A csv.gz -m -p -E -k -K -np -nd \
https://physionet.org/works/MIMICIIIClinicalDatabase/files/

Following https://mimic.physionet.org/tutorials/install-mimic-locally-ubuntu/

In [13]:
%%bash

# start postgresql server
brew services restart postgresql

==> Successfully started `postgresql` (label: homebrew.mxcl.postgresql)


In [10]:
# specify database using sqlalchemy
db_string = 'postgresql://ruoyijiang:@localhost/mimic'

In [11]:
# create database using default public schema
engine = create_engine(db_string)
if not database_exists(engine.url):
    create_database(engine.url)

In [12]:
# reconnect to database under public schema
engine = create_engine(db_string,
    connect_args={'options': '--search_path={}'.format('public')})

In [13]:
# create tables
# %%bash
# psql 'dbname=mimic user=ruoyijiang options=--search_path=public' \
# -f ~/git/mimic-code/buildmimic/postgres/postgres_create_tables.sql 

In [14]:
# load tables (check script for gunzip)
# %%bash
# psql 'dbname=mimic user=ruoyijiang options=--search_path=public' \
# ~/git/mimic-code/buildmimic/postgres/postgres_load_data.sql 
# remember to set... mimic_data_dir='/Users/ruoyijiang/Projects/Haimovich_Pressors/collection/mimic'

In [15]:
connection = engine.connect()

In [21]:
# create indexes using sql file
sql_command_file = os.path.join(os.getenv("HOME"),'git/mimic-code/buildmimic/postgres/postgres_add_indexes.sql')

result = connection.execute(text(createCommandSQL(sql_command_file)))

In [ ]:
# checks
sql_command_file = os.path.join(os.getenv("HOME"),'git/mimic-code/buildmimic/postgres/postgres_checks.sql')

result = connection.execute(text(createCommandSQL(sql_command_file)))

### Commandline terminal queries (if needed)

In [ ]:
# connect to database for interactive
# psql 'dbname=mimic user=ruoyijiang options=--search_path=public'

In [ ]:
# Print all tables in schema
#\d+

# Print metadata on admissions
#\d+ admissions

### Querying within postgres

In [90]:
# How to count?
sql_query_text = \
"SELECT COUNT(*) \
FROM patients;"

pd.read_sql_query(sql_query_text, engine)

,count
0,46520


In [24]:
# How to unique?
sql_query_text = \
"SELECT DISTINCT(gender) \
FROM patients;"

pd.read_sql_query(sql_query_text, engine)

,gender
0,M
1,F


In [64]:
# How to filter and count?
sql_query_text = \
"SELECT COUNT(*) \
FROM patients \
WHERE gender = 'F';"

pd.read_sql_query(sql_query_text, engine)

,count
0,20399


In [25]:
# How to group_by and count?
sql_query_text = \
'SELECT gender, COUNT(*) \
FROM patients \
GROUP BY gender;'

pd.read_sql_query(sql_query_text, engine)

,gender,count
0,M,26121
1,F,20399


In [74]:
sql_query_text

'SELECT gender, COUNT(*) FROM patients GROUP BY gender;'

In [26]:
# How to inner join? (and mutate)?

sql_query_text = \
"SELECT p.subject_id, p.dob, a.hadm_id, a.admittime, p.expire_flag, \
    MIN (a.admittime) OVER (PARTITION BY p.subject_id) AS first_admittime \
FROM admissions a \
INNER JOIN patients p \
ON p.subject_id = a.subject_id \
ORDER BY a.hadm_id, p.subject_id;"

pd.read_sql_query(sql_query_text, engine).head()

,subject_id,dob,hadm_id,admittime,expire_flag,first_admittime
0,58526,2082-03-21,100001,2117-09-11 11:46:00,0,2117-09-11 11:46:00
1,54610,2090-05-19,100003,2150-04-17 15:34:00,1,2150-04-17 15:34:00
2,9895,2059-05-07,100006,2108-04-06 15:49:00,1,2108-04-06 15:49:00
3,23018,2071-06-04,100007,2145-03-31 05:33:00,0,2145-03-31 05:33:00
4,533,2101-07-30,100009,2162-05-16 15:56:00,0,2162-05-16 15:56:00


In [27]:
sql_query_text = \
"WITH first_admission_time AS \
( \
SELECT \
    p.subject_id, p.dob, p.gender \
    , MIN (a.admittime) AS first_admittime \
    , MIN( ROUND( (cast(admittime as date) - cast(dob as date)) / 365.242,2) ) \
    AS first_admit_age \
    FROM patients p \
    INNER JOIN admissions a \
    ON p.subject_id = a.subject_id \
    GROUP BY p.subject_id, p.dob, p.gender \
    ORDER BY p.subject_id \
) \
SELECT \
subject_id, dob, gender \
, first_admittime, first_admit_age \
, CASE \
WHEN first_admit_age > 89 \
then '>89' \
WHEN first_admit_age >= 14 \
THEN 'adult' \
WHEN first_admit_age <= 1 \
THEN 'neonate' \
ELSE 'middle' \
END AS age_group \
FROM first_admission_time \
ORDER BY subject_id"

pd.read_sql_query(sql_query_text, engine).head()

,subject_id,dob,gender,first_admittime,first_admit_age,age_group
0,2,2138-07-17,M,2138-07-17 19:04:00,0.00,neonate
1,3,2025-04-11,M,2101-10-20 19:08:00,76.52,adult
2,4,2143-05-12,F,2191-03-16 00:28:00,47.84,adult
3,5,2103-02-02,M,2103-02-02 04:31:00,0.00,neonate
4,6,2109-06-21,F,2175-05-30 07:15:00,65.94,adult


### Extract pivoted_vital from chartevents

In [4]:
%%bash
psql 'dbname=mimic user=ruoyijiang options=--search_path=public' \
-f ~/git/mimic-code/concepts/pivot/pivoted-vital.sql

DROP MATERIALIZED VIEW
SELECT 9160700


In [21]:
sql_query_text = \
"SELECT * FROM pivoted_vital LIMIT 10;"

pd.read_sql_query(sql_query_text, engine)

,icustay_id,charttime,heartrate,sysbp,diasbp,meanbp,resprate,tempc,spo2,glucose
0,200001,2181-11-25 19:06:00,115.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,200001,2181-11-25 19:07:00,NaN,NaN,NaN,NaN,22.0,NaN,NaN,NaN
2,200001,2181-11-25 19:08:00,NaN,113.0,65.0,77.0,NaN,NaN,NaN,NaN
3,200001,2181-11-25 19:14:00,NaN,NaN,NaN,NaN,NaN,NaN,94.0,NaN
4,200001,2181-11-25 19:16:00,114.0,NaN,NaN,NaN,26.0,37.277778,95.0,NaN
5,200001,2181-11-25 20:00:00,113.0,110.0,65.0,76.0,20.0,NaN,97.0,NaN
6,200001,2181-11-25 21:00:00,108.0,113.0,68.0,79.0,18.0,NaN,98.0,NaN
7,200001,2181-11-25 22:00:00,110.0,116.0,68.0,79.0,27.0,NaN,98.0,118.0
8,200001,2181-11-25 23:00:00,102.0,102.0,61.0,71.0,21.0,37.666667,96.0,NaN
9,200001,2181-11-26 00:00:00,108.0,NaN,NaN,NaN,19.0,NaN,94.0,NaN


In [17]:
sql_query_text = \
"SELECT COUNT(*) FROM pivoted_vital;"

pd.read_sql_query(sql_query_text, engine).head()

,count
0,9160700


In [34]:
sql_query_text = \
"SELECT * FROM pivoted_vital;"

vitals_df =  pd.read_sql_query(sql_query_text, engine)

In [36]:
vitals_df.shape

(9160700, 10)

In [35]:
vitals_df.to_csv(os.path.join(save_dir, "vitals.csv"))

In [85]:
os.path.join(save_dir, "vitals.csv")

'/Users/ruoyijiang/Projects/Haimovich_Pressors/collection/out/vitals.csv'

### Extract pressors from inputevents_cv

In [50]:
duration_commands = \
    [command for command in os.listdir(os.path.join(os.getenv("HOME"),"git/mimic-code/concepts/durations")) if 'durations' in command]

In [68]:
for duration_command in duration_commands:
    try:
        cmd = [
        "psql 'dbname=mimic user=ruoyijiang options=--search_path=public'",
        "-f", os.path.join(os.getenv("HOME"),"git/mimic-code/concepts/durations", duration_command)
        ]

        result = subprocess.Popen(shlex.split(' '.join(cmd)), stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        output = result.communicate()
        print(duration_command)
        print(output[0].decode("utf-8") + "\nERROR: \n" + output[1].decode("utf-8"))
    except:
        print(duration_command, " not run")

epinephrine-durations.sql
DROP MATERIALIZED VIEW
SELECT 3126

ERROR: 

phenylephrine-durations.sql
DROP MATERIALIZED VIEW
SELECT 33141

ERROR: 
psql:/Users/ruoyijiang/git/mimic-code/concepts/durations/phenylephrine-durations.sql:5: NOTICE:  materialized view "phenylephrinedurations" does not exist, skipping

arterial-line-durations.sql
DROP MATERIALIZED VIEW
SELECT 30424

ERROR: 
psql:/Users/ruoyijiang/git/mimic-code/concepts/durations/arterial-line-durations.sql:1: NOTICE:  materialized view "arterial_line_durations" does not exist, skipping

dobutamine-durations.sql
DROP MATERIALIZED VIEW
SELECT 1792

ERROR: 
psql:/Users/ruoyijiang/git/mimic-code/concepts/durations/dobutamine-durations.sql:5: NOTICE:  materialized view "dobutaminedurations" does not exist, skipping

vasopressin-durations.sql
DROP MATERIALIZED VIEW
SELECT 4190

ERROR: 
psql:/Users/ruoyijiang/git/mimic-code/concepts/durations/vasopressin-durations.sql:5: NOTICE:  materialized view "vasopressindurations" does not exist,

In [69]:
sql_query_text = "SELECT * FROM pg_class;"

all_dbs = pd.read_sql_query(sql_query_text, engine)

duration_tables = [duration for duration in all_dbs.relname if 'duration' in duration]

In [70]:
duration_tables

['arterial_line_durations',
 'dobutaminedurations',
 'crrtdurations',
 'vasopressindurations',
 'dopaminedurations',
 'central_line_durations',
 'milrinonedurations',
 'epinephrinedurations',
 'vasopressordurations',
 'phenylephrinedurations',
 'ventdurations',
 'isupreldurations',
 'norepinephrinedurations',
 'adenosinedurations']

In [76]:
pressor_dict = {}
for duration_table in duration_tables:
    sql_query_text = \
    "SELECT * FROM " + duration_table + ";"

    pressor_df = pd.read_sql_query(sql_query_text, engine)
    pressor_df['pressor'] = duration_table
    
    pressor_dict[duration_table] = pressor_df

In [79]:
pressor_total_df = pd.concat(pressor_dict, axis = 0)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [84]:
pressor_total_df.reset_index().to_csv(os.path.join(save_dir, "pressors.csv"))